In [1]:
from datetime import datetime, timedelta 
import yfinance as yf
import pandas as pd
import pandas_gbq
from google.cloud import bigquery
import os
import json

In [2]:
# Change to your key
key = 'C:/AA NUS/Y3S2/IS3107/Project Testing/key.json'

In [3]:
'''
Extract stock information in STI Components for past 2 months
Gets the stock data using Yahoo Finance API in Pandas Dataframe and push as JSON

Input: None
Output: None
'''
#STI Index
tickers =  ['D05.SI', 'SE', 'O39.SI', 'U11.SI' ,'Z74.SI', 'F34.SI', 'C6L.SI', 'GRAB', 'G13.SI', 'C38U.SI','G07.SI', 'C07.SI', 'A17U.SI', 'S63.SI', 'BN4.SI']
i = 0
df = pd.DataFrame()

for ticker in tickers:
    prices = yf.download(ticker, start="2023-02-18", end="2023-04-19").iloc[: , :6].dropna(axis=0, how='any')
    prices = prices.loc[~prices.index.duplicated(keep='last')]
    prices = prices.reset_index()
    print(prices)
    prices.insert(loc = 1, column = 'Ticker', value = ticker)
    prices = prices.rename({'Adj Close': 'Adj_Close'}, axis=1)
    df = pd.concat([df, prices],ignore_index = True)
    i += 1

stock_info_daily = df.to_json(orient='records')
print(stock_info_daily)

[*********************100%***********************]  1 of 1 completed
         Date       Open       High        Low      Close  Adj Close   Volume
0  2023-02-20  35.250000  35.250000  34.910000  34.919998  34.396931  1978300
1  2023-02-21  35.080002  35.080002  34.770000  34.860001  34.337833  2353400
2  2023-02-22  34.950001  35.000000  34.619999  34.660000  34.140827  3618000
3  2023-02-23  34.849998  34.849998  34.340000  34.340000  33.825623  3408600
4  2023-02-24  34.340000  34.680000  34.330002  34.400002  33.884724  3871293
5  2023-02-27  34.299999  34.590000  34.169998  34.340000  33.825623  3582900
6  2023-02-28  34.470001  34.490002  34.139999  34.169998  33.658165  3463200
7  2023-03-01  34.209999  34.349998  34.000000  34.000000  33.490715  3394723
8  2023-03-02  33.860001  34.020000  33.500000  33.549999  33.047455  4763399
9  2023-03-03  33.549999  33.740002  33.480000  33.500000  32.998203  4503800
10 2023-03-06  33.689999  33.869999  33.509998  33.820000  33.313408  292

In [4]:
'''
Push the raw stockdata into google bigquery dataset yfinance_data.stock_raw
'''
df = pd.DataFrame(eval(stock_info_daily))
df['Date'] = pd.to_datetime(df['Date'], unit='ms') # Convert date from json to date format

openfile = open(key)
jsondata = json.load(openfile)
openfile.close()

# Construct a BigQuery client object.
credentials_path = key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
client = bigquery.Client()

project_id = jsondata['project_id']
staging_table_id = "yfinance_data.stock_raw"
pandas_gbq.to_gbq(df, destination_table=staging_table_id, 
                    project_id=project_id,
                    if_exists='append')

100%|██████████| 1/1 [00:00<?, ?it/s]


In [6]:
'''
Transform stockdata and push into google bigquery dataset yfinance_data.stock_info
'''
openfile = open(key)
jsondata = json.load(openfile)
openfile.close()
project_id = jsondata['project_id']
staging_table_id = project_id + ".yfinance_data.stock_raw"
actual_table_id = project_id + ".yfinance_data.stock_info"

#Connect To Bigquery
credentials_path = key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
client = bigquery.Client()

#Load Data from Staging table to Main table
query =  f"""
    INSERT INTO `{actual_table_id}`
    SELECT *
    FROM (SELECT *, AVG(CAST(Close AS float64)) OVER (PARTITION BY p.Ticker ORDER BY Date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) AS MA_5day,
    CASE
    WHEN ((Close - AVG(CAST(Close AS float64)) OVER (PARTITION BY p.Ticker ORDER BY Date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW)) > 0.1) or (AVG(CAST(Close AS float64)) OVER (PARTITION BY p.Ticker ORDER BY Date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) - Close) > 0.1 THEN 'Neutral'
    WHEN Close > AVG(CAST(Close AS float64)) OVER (PARTITION BY p.Ticker ORDER BY Date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) THEN 'Buy'
    WHEN Close < AVG(CAST(Close AS float64)) OVER (PARTITION BY p.Ticker ORDER BY Date ROWS BETWEEN 4 PRECEDING AND CURRENT ROW) THEN 'Sell'
    else 'Neutral'
    END AS Signal,
    FROM
    `{staging_table_id}` as p
    ) T;

    TRUNCATE TABLE `{staging_table_id}`;
"""
query_job = client.query(query)
# query_job = client.query(query, job_config=bigquery.QueryJobConfig(write_disposition="WRITE_TRUNCATE"))
print('Successfully loaded stock prices')

Successfully loaded stock prices


In [7]:
'''
Load stockdata and into combined_data with twitter sentiment scores
'''
openfile = open(key)
jsondata = json.load(openfile)
openfile.close()
project_id = jsondata['project_id']
finance_table_id = project_id + ".yfinance_data.stock_info"
twitter_table_id = project_id + ".twitter_data.stock_aggregated"
load_table_id = project_id + ".combined_data.stock_info"

#Connect To Bigquery
credentials_path = key
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = credentials_path
client = bigquery.Client()

#Joining finance table and twitter scores table per day
query = f"""
    INSERT INTO `{load_table_id}`
    SELECT 
        A.Date,
        A.Ticker,
        A.Open,
        A.High,
        A.Low,
        A.Close,
        A.Adj_Close,
        A.Volume,
        A.MA_5days,
        A.Signal,
        B.WeightedCompoundScore
    FROM
        `{finance_table_id}` as A

    JOIN `{twitter_table_id}` as B ON A.Ticker = B.Ticker AND A.Date = B.Date
"""
query_job = client.query(query)
# query_job = client.query(query, job_config=bigquery.QueryJobConfig(write_disposition="WRITE_TRUNCATE"))
print('Successfully loaded combined table')

Successfully loaded combined table
